<a href="https://colab.research.google.com/github/jodfedlet/TCC/blob/main/TCCII/Twitter/LDA_with_gensim_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim
!pip install pyldavis
! pip install bs4
! pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:

import pandas as pd
import re
import json
import gzip
import gensim
import contractions
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount("/content/drive")

from pprint import pprint
from gensim import corpora
from bs4 import BeautifulSoup
from gensim.models import Phrases
from gensim.models import LdaModel

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
try:
 dftweets = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TCC/csv/pre_processed_tweets.csv', lineterminator='\n', index_col=0)
except:
  print("Não foi possível carregar o arquivo")

In [4]:
dftweets.head()

,datetime,tweet_id,user_id,text,count_word,text_preprocessed,text_lematizados,bigrams_text
1,2022-05-30 23:00:53+00:00,1531410348141592577,1473798922472767488,@BetoLuiz_RU @jonesmanoel_PCB @deccache O Hait...,15,haiti desgraca fato erros anulam acertos fala ...,haiti desgraca fato erro anular acerto falir s...,haiti desgraca fato erro anular acerto falir s...
2,2022-05-30 22:57:20+00:00,1531409451122577410,1521912766151417856,O senhor vai investir na educação da Venezuela...,16,senhor vai investir educacao venezuela argenti...,senhor ir investir educacao venezuela argentin...,senhor ir investir_educacao venezuela argentin...
3,2022-05-30 22:55:14+00:00,1531408925945434112,1253882714723409920,"@rsallesmma Que nada, na mão de vocês já tá vi...",6,nada mao voces virando mistura deserto saara h...,nado mao voces virar misturar desertar saara h...,nado mao voces virar_misturar desertar_saara h...
4,2022-05-30 22:55:14+00:00,1531408923277807617,40320768,@JoppertLeonardo @jonesmanoel_PCB @deccache En...,22,entramos brics invadimos haiti mando bush lula...,entrar brics invadir haiti mandar bush lula di...,entrar brics invadir haiti mandar bush lula di...
5,2022-05-30 22:37:49+00:00,1531404543568781313,1082697350441824257,"@alice_mader9 kkkkkk e era 2 haitiano, n enten...",3,haitiano entendi nada falavam,haitiano entender nado falar,haitiano entender_nado falar


In [5]:
dftweets.bigrams_text.head().tolist()

['haiti desgraca fato erro anular acerto falir sobrar guerra_ucrania vir posicionar bem criticar posturar zelensky',
 'senhor ir investir_educacao venezuela argentino haiti cubar etc paises comunista porque aqui brasil senhor nunca fazer',
 'nado mao voces virar_misturar desertar_saara haiti',
 'entrar brics invadir haiti mandar bush lula dizer ir fortalecer brics fazer palestrar parlamentar europeu este tratar russia_chino inimigo mortal ir funcionar',
 'haitiano entender_nado falar']

In [6]:
#agrupar df por mês e adicionar os períodos numa lista
dftweets['datetime'] = pd.to_datetime(dftweets.datetime, format='%Y-%m-%d %H:%M:%S')
df_by_year_month = [df_year_month for df_year_month in dftweets.groupby([dftweets['datetime'].dt.year.rename('Year'), dftweets['datetime'].dt.month.rename('Month')])]

In [7]:
#2018-01 até 2022-05
len(df_by_year_month)

53

In [8]:
def get_tweets_period(df_tuple):
  #idenficar o peródo dos tweets
  m = df_tuple[0][1]
  month = '0'+ str(m) if m < 10 else m
  return str(df_tuple[0][0]) +'-'+str(month)

###gensim oficial doc 
https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py

In [19]:
k_topicos = [3, 5, 9, 15, 25]

In [50]:
for df_y_m in df_by_year_month:
  #df period and mini stats
  period = get_tweets_period(df_y_m)
  print(f'Período: {period}')
  words_count = df_y_m[1]['count_word'].sum()
  docs = df_y_m[1].bigrams_text.tolist()
  tam_doc = len(docs)
  # print(f'Tamanho do documento: {tam_doc} tweets, {words_count} palavras e {round(words_count/tam_doc, 2)} de médias de palavras')
  # print('-'*80)

  # print(docs)
  # break

  docs_list = [[word for word in doc.split()] for doc in docs]
  #docs_list = docs_list[0:3]
  #criando o dicionário da representação dos documentos
  dictionary = corpora.Dictionary(docs_list)
  #criando o corpus (bags-of-words dos documentos)
  corpus = [dictionary.doc2bow(text) for text in docs_list]

  # print(f'Number of unique tokens: {len(dictionary)}')
  # print(f'Number of documents: {len(corpus)}')

  for k_topico in k_topicos:
    LDA = LdaModel(
        corpus=corpus, 
        id2word=dictionary, 
        num_topics=k_topico, 
        chunksize = tam_doc,
        alpha='auto',
        eta='auto',
        iterations=400,
        passes=10,
        eval_every=None
    )

    top_topics = LDA.top_topics(corpus)

    print(f'Para {k_topico} tópicos -------------')
    for topic in top_topics:
      topic_list = [word[1] for word in topic[0]]
      print(topic_list)
      # print(topic[0])
      # print(list(topic[0]))
      # break

    # break
    # print(len(top_topics))
    #pprint(top_topics)

    print('*'*200)
    print()
    #break
  # break

  

Período: 2018-01
Para 3 tópicos -------------
['haiti', 'nao', 'haitiano', 'trump', 'merda', 'pai', 'dizer', 'sobrar', 'brasil', 'pra', 'fazer', 'paises', 'ir', 'falar', 'ser', 'africa', 'presidente', 'todo', 'pessoa', 'eua']
['haiti', 'nao', 'haitiano', 'trump', 'pra', 'aqui', 'ver', 'pai', 'eua', 'sobrar', 'fazer', 'dizer', 'ir', 'poder', 'achar', 'saber', 'paises', 'brasil', 'presidente', 'merda']
['haiti', 'haitiano', 'nao', 'trump', 'pai', 'paises', 'pra', 'querer', 'ver', 'africa', 'merda', 'fazer', 'ir', 'dizer', 'brasil', 'chamar', 'caro', 'sao', 'falar', 'aqui']
********************************************************************************************************************************************************************************************************

Para 5 tópicos -------------
['haiti', 'nao', 'paises', 'merda', 'haitiano', 'trump', 'brasil', 'pessoa', 'pai', 'presidente', 'paises_africano', 'salvador', 'sobrar', 'imigrante', 'ser', 'querer', 'ver', 'dizer', 'fazer